# Evaluation Model 4
## Model name : unsloth/llama-3-8b-unsloth
## Link Colab : [Click](https://colab.research.google.com/drive/1tcBafuKvZaHrb6qUIw016SaWDVRF3oiU?usp=sharing)
## Link Adapter weight : [Click](https://drive.google.com/file/d/1sqVp1FinYwrqss-S1i6chSK2r9UN9Jx0/view?usp=drive_link)

# Download Dependencies

In [ ]:
%%capture
! pip install unsloth
! pip install rouge_score
! pip install evaluate

In [ ]:
%%capture
! pip install -U peft
! pip install bert_score

# Import Dependencies

In [ ]:
from tqdm import tqdm
import kagglehub
import os

In [ ]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
import evaluate

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
from datasets import load_dataset
import evaluate
import torch
import bert_score
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import os
import re

In [ ]:
from datasets import load_dataset, DatasetDict
from sklearn.model_selection import train_test_split
import pandas as pd
from datasets import Dataset

In [ ]:
from peft import PeftModel

# Load dataset

In [ ]:
# 1. Load dataset
dataset = load_dataset("farwew/DoctorsAnswerTextDataset-in-Indonesian")

In [ ]:
# 3. Ambil data test
dataset_eval = dataset["test"]

# 4. Konversi ke pandas DataFrame
df = dataset_eval.to_pandas()
df.head()

,instruction,input,output,topic_set,topics
0,Selalu tersedak setiap makan dan minum,Selalu tersedak setiap makan dan minum. Dok sa...,batuk adalah respon tubuh untuk mengeluarkan b...,gastritis,penyakit-parkinson multiple-sclerosis gastritis
1,Apa obat untuk demam?,Apa obat untuk demam?. malam dok saya mau tany...,"kepala pusing, perut mual, punggung sakit dan ...",mual,flu pilek sakit-kepala nyeri-punggung mual
2,Cara mengatasi perut kembung pada anak,Cara mengatasi perut kembung pada anak. punya ...,perut kembung menandakan adanya penumpukan gas...,perut-kembung,anak gastritis perut-kembung
3,Perut Bagian Bawah Kembung,"Perut Bagian Bawah Kembung. saya mau bertanya,...","seperti: sayur: kol, brokoli, kembang kol, asp...",gangguan-pencernaan,gangguan-pencernaan
4,Nyeri dada menjalar ke punggung,Nyeri dada menjalar ke punggung. Malam dok tad...,saya coba bantu jawab. keluhan nyeri dada menj...,sakit-maag,nyeri-dada sakit-maag


In [ ]:
# 5. Stratified split berdasarkan kolom 'topic_set'
train_df, stratified_sample_df = train_test_split(
    df,
    test_size=100,  # ubah sesuai proporsi yang diinginkan
    stratify=df["topic_set"],
    random_state=42
)

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
stratified_dataset = Dataset.from_pandas(stratified_sample_df)

In [ ]:
stratified_dataset

Dataset({
    features: ['instruction', 'input', 'output', 'topic_set', 'topics', '__index_level_0__'],
    num_rows: 100
})

# Load Adapter

In [ ]:
%%capture
!pip install gdown

In [ ]:
# ! gdown 'https://drive.google.com/uc?id=1sqVp1FinYwrqss-S1i6chSK2r9UN9Jx0'

Downloading...
From (original): https://drive.google.com/uc?id=1sqVp1FinYwrqss-S1i6chSK2r9UN9Jx0
From (redirected): https://drive.google.com/uc?id=1sqVp1FinYwrqss-S1i6chSK2r9UN9Jx0&confirm=t&uuid=3ddf62d4-a6d0-402e-89f1-aa4e9648fc39
To: /content/1 epoch -  Mistral_v0.3_ instruct (7B) - Alpaca (run 3).zip
100% 78.4M/78.4M [00:00<00:00, 122MB/s]


In [ ]:
# ! unzip "/content/1 epoch -  Mistral_v0.3_ instruct (7B) - Alpaca (run 3).zip"

#  Notebook Evaluasi Base Model (tanpa Adapter LoRA)

In [ ]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSN

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.

### Input:
{}

### Response:
{}"""

In [ ]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for input_text, output_text in zip(inputs, outputs):
        text = alpaca_prompt.format(input_text, output_text) + EOS_TOKEN
        texts.append(text)
    return { "text": texts }
pass

## Inference dengan base model

In [ ]:
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "mencegah mual",
            ""
        )
    ],
    return_tensors="pt"
).to("cuda")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=True,
        top_p=0.9,
        temperature=0.7,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.

### Input:
mencegah mual

### Response:
Untuk mencegah mual, Anda harus minum banyak air. Anda juga harus makan makanan yang ringan dan tidak mengandung lemak. Jika Anda makan makanan yang mengandung lemak, itu akan membuat Anda mual.


## Evaluation

In [ ]:
# === Evaluasi === #
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

generated_outputs = []
reference_outputs = []
all_instructions = []
all_inputs = []

In [ ]:
def normalize_text(text):
    text = text.lower().strip()
    text = re.sub(r"\p{P}+", "", text)
    text = re.sub(r"\s+", " ", text)
    return text

In [ ]:
num = 1
for example in tqdm(stratified_dataset, desc="Evaluating Base Model"):
    # Gunakan prompt yang hanya menerima input (karena instruksi tetap)
    prompt = alpaca_prompt.format(example["input"], "")  # output dikosongkan untuk inferensi

    # Tokenisasi dan pindahkan ke GPU
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Inference (tanpa perhitungan gradien)
    with torch.no_grad():
        output = model.generate(
            **input_ids,
            max_new_tokens=1024,
            do_sample=True,
            top_p=0.9,
            temperature=0.7
        )

    # Decode output dan ekstrak bagian response-nya
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    response = decoded.split("### Response:")[-1].strip()

    # Cetak sampel jika masih kurang dari 10
    if num < 10:
        print(prompt)
        print()
        print('response:', response)
        print('-' * 80)

    # Simpan hasil untuk evaluasi
    generated_outputs.append(response)
    reference_outputs.append(example["output"])
    all_instructions.append("Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.")  # atau bisa juga diabaikan jika instruksi selalu sama
    all_inputs.append(example["input"])
    num += 1

Evaluating Base Model:   1%|          | 1/100 [01:06<1:49:34, 66.41s/it]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.

### Input:
Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1,5 kilo, tidak memiliki nafsu makan sama sekali, sering sendawa, nyeri di ulu hati dan perut bagian kanan. Kemarin saya juga sempat terlambat makan siang dan ma

Evaluating Base Model:   2%|▏         | 2/100 [02:09<1:44:58, 64.27s/it]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.

### Input:
solusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?

### Response:


response: Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.

### Input:
solusi untuk mengat
--------------------------------------------------------------------------------


Evaluating Base Model:   3%|▎         | 3/100 [02:42<1:21:05, 50.16s/it]

...

Evaluating Base Model:   8%|▊         | 8/100 [06:02<56:41, 36.97s/it]  

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.

### Input:
sakit perut sebelah kiri bawah. Dok, saya mau tanya. Saya sudah menikah dan berusia 21 tahun. Sudah hampir 2minggu lebih perut sebelah kiri bawah sakit. Terhitung dari selesai saya haid tanggal 30 maret 2016. Sakitnya sampai ke pinggang dan tanda tanda nya seperti akan haid. Karena pada saya pun terasa nyeri dan cenderung lebih sensitif. Di minggu ketiga ini sakitnya terasa ngilu sampai ke paha kiri. Bahkan kadang panggul panggul pun sakit. Itu kenapa ya dok?apakah saya hamil? Terima kasih

### Response:


response: saya mau tanya. Saya sudah menikah dan berusia 21 tahun. Sudah hampir 2minggu lebih perut sebelah kiri bawah sakit. Terhitung dari sel

Evaluating Base Model:   9%|▉         | 9/100 [06:14<44:24, 29.29s/it]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.

### Input:
Nyeri Ulu Hati Pasca Operasi Usus buntu. sudah 4 Bulan yang lalu saya di operasi usus buntu (Operasi Laparotomi) minggu minggu ini perut saya aga kurang enak dok. tadi pagi setelah BAB perut saya kram,melilit,perih....dan nyeri Ulu ati, nyerinya timbul tenggelam...tapi setelah minum obat maag (Omeprazol) sakitnya mereda tapi masih aga kurang enak perutnya itu penyebabnya apa ya dok? apa karena Maag atau ada iritasi di Usus? atau ada hubungannya dengan operasi usus buntu 4 bulan yang lalu?

### Response:


response: Saya telah mempelajari dokumen medis Anda. Terima kasih telah memilih kami untuk melayani kebutuhan kesehatan Anda. Saya memiliki beber

Evaluating Base Model: 100%|██████████| 100/100 [53:14<00:00, 31.95s/it]


In [ ]:
# === Skor Evaluasi === #
P, R, F1 = bert_score.score(generated_outputs, reference_outputs, lang="id", device="cuda")
bleu_score = bleu.compute(predictions=generated_outputs, references=[[r] for r in reference_outputs])
rouge_score = rouge.compute(predictions=generated_outputs, references=reference_outputs)

timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
output_dir = f"eval_basemodel_{timestamp}"
os.makedirs(output_dir, exist_ok=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [ ]:
# === Simpan Hasil Evaluasi === #
df = pd.DataFrame({
    "instruction": all_instructions,
    "input": all_inputs,
    "reference": reference_outputs,
    "generated": generated_outputs,
    "bert_precision": P.tolist(),
    "bert_recall": R.tolist(),
    "bert_f1": F1.tolist(),
})

df.to_csv(f"{output_dir}/base_model_outputs.csv", index=False)

metrics = {
    "bert_score": {
        "precision": round(P.mean().item() * 100, 2),
        "recall": round(R.mean().item() * 100, 2),
        "f1": round(F1.mean().item() * 100, 2)
    },
    "bleu": round(bleu_score["bleu"] * 100, 2),
    "rouge": {
        "rouge1": round(rouge_score["rouge1"] * 100, 2),
        "rouge2": round(rouge_score["rouge2"] * 100, 2),
        "rougeL": round(rouge_score["rougeL"] * 100, 2)
    }
}

with open(f"{output_dir}/metrics.json", "w") as f:
    import json; json.dump(metrics, f, indent=4)

print("=== EVALUASI BASE MODEL SELESAI ===")
print(metrics)

=== EVALUASI BASE MODEL SELESAI ===
{'bert_score': {'precision': 62.21, 'recall': 59.62, 'f1': 60.81}, 'bleu': 0.68, 'rouge': {'rouge1': np.float64(12.27), 'rouge2': np.float64(1.3), 'rougeL': np.float64(7.67)}}


In [ ]:
hasil_base = pd.read_csv(f"{output_dir}/base_model_outputs.csv")
hasil_base

,instruction,input,reference,generated,bert_precision,bert_recall,bert_f1
0,Jawab pertanyaan medis berikut dengan bahasa y...,Solusi atasi sakit maag yang tak kunjung sembu...,usus halus ataupun kerongkongan yang dapat dis...,"Maaf, dokter tidak tersedia pada hari ini. Ber...",0.705103,0.661096,0.682391
1,Jawab pertanyaan medis berikut dengan bahasa y...,solusi untuk mengatasi konstipasi. dok kalo ba...,konstipasi adalah keadaan tidak dapat buang ai...,Jawab pertanyaan medis berikut dengan bahasa y...,0.644507,0.582390,0.611876
2,Jawab pertanyaan medis berikut dengan bahasa y...,"Perut sakit, mual dan bersendawa berlebihan. s...","penyebab sakit perut, mual dan bersendawa mung...","Assalamu'alaikum,\n\nPerut sakit, mual dan ber...",0.634713,0.627972,0.631325
3,Jawab pertanyaan medis berikut dengan bahasa y...,Perut sering sakit di sertai mual. Maaf dok' k...,sebaiknya anda berkonsultasi dengan dokter di ...,Jawab pertanyaan medis berikut dengan bahasa y...,0.661312,0.605074,0.631944
4,Jawab pertanyaan medis berikut dengan bahasa y...,Pengobatan pada alaergi makanan disertai sakit...,alergi adalah kondisi kelainan sistem kekebala...,Kondisi alergi makanan seafood dapat ditangani...,0.642492,0.633456,0.637942
...,...,...,...,...,...,...,...
95,Jawab pertanyaan medis berikut dengan bahasa y...,Muntah-muntah pada penderita infeksi lambung a...,lambung adalah organ yang vital fungsinya dala...,Muntah yang tidak disengaja adalah gejala yang...,0.560947,0.531560,0.545859
96,Jawab pertanyaan medis berikut dengan bahasa y...,"Demam tinggi, mual, dan tidak nafsu makan. do...","kolera, disentri dulu. radang usus = kolitis ,...",Anda mungkin mengalami penyakit maag akut. Maa...,0.631387,0.639182,0.635261
97,Jawab pertanyaan medis berikut dengan bahasa y...,Perbedaan serangan jantung dan lambung. Dok ma...,"namun, untuk membedakan apakah serangan jantun...","Dok, angin masuk itu bisa disebabkan karena ma...",0.580312,0.598176,0.589109
98,Jawab pertanyaan medis berikut dengan bahasa y...,Penyebab BAB berdarah dan mengenai pemeriksaan...,anal seks merupakan salah satu perbuatan beres...,"> *Dear patient,*\n> \n> *Thank you for reachi...",0.613508,0.575765,0.594037


In [ ]:
# prompt: ! zip /content/eval_basemodel_20250525_034008 Base_model(4).zip

! zip -r /content/eval_basemodel_4 /content/eval_basemodel_20250525_034008

  adding: content/eval_basemodel_20250525_034008/ (stored 0%)
  adding: content/eval_basemodel_20250525_034008/metrics.json (deflated 46%)
  adding: content/eval_basemodel_20250525_034008/base_model_outputs.csv (deflated 77%)


# Dengan Adapter

In [ ]:
import torch
import gc

# Clear PyTorch cache
torch.cuda.empty_cache()

# Garbage collection
gc.collect()

4

In [ ]:
model = PeftModel.from_pretrained(model, "farwew/DoctorsAnswerTextDataset-in-IndonesianUnsloth-llama-3-8b-bnb-4bit")

In [ ]:
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
    "mencegah mual",
    "Anak saya mengalami mual, bagaimana mencegah mual?",
    ""
)
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 512)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.

### Input:
mencegah mual

### Response:
Anak saya mengalami mual, bagaimana mencegah mual? terima kasih dokter sebelumnya, saya akan menjawab pertanyaan anda. mual merupakan suatu kondisi dimana terjadi sensasi tidak nyaman pada perut, yang disertai dengan rasa ingin muntah. mual pada anak dapat disebabkan oleh beberapa hal berikut ini: infeksi saluran pencernaan, seperti karena infeksi virus, bakteri, atau parasit. infeksi saluran pencernaan dapat menyebabkan gejala mual, muntah, demam, lemas, diare, atau konstipasi. infeksi saluran pencernaan dapat disebabkan oleh banyak hal, seperti karena makan makanan yang tidak bersih, kebersihan makana

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
    "Selalu tersedak setiap makan dan minum",
    "Selalu tersedak setiap makan dan minum. Dok saya mau tanya... Kenapa kalau saya makan atau minum selalu tersedak.. Akhirnya susah nafas.. Dada sangat sesak.... Hampir setiap makan saya tersedak...",
    ""
)
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 512)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.

### Input:
Selalu tersedak setiap makan dan minum

### Response:
Selalu tersedak setiap makan dan minum. Dok saya mau tanya... Kenapa kalau saya makan atau minum selalu tersedak.. Akhirnya susah nafas.. Dada sangat sesak.... Hampir setiap makan saya tersedak... Kira kira saya sakit apa ya dok... Mohon penjelasannya... Terima kasih

penyebab tersedak saat makan atau minum dapat disebabkan oleh berbagai hal, diantaranya : penyakit asam lambung gangguan pada tenggorokan penyakit asam lambung atau gerd merupakan suatu keadaan dimana asam lambung mengalir balik ke kerongkongan. hal ini disebabkan oleh tidak berfungsinya lower esophageal sphincter 

In [ ]:
# === IMPORT TAMBAHAN === #
import bert_score
import pandas as pd
import json
from datetime import datetime
import os

# === LOAD METRIK EVALUASI === #
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

generated_outputs = []
reference_outputs = []
all_inputs = []  # Untuk menyimpan input
all_instructions = []  # Untuk menyimpan instruction

# === PARAMETER === #
batch_size = 1  # Sesuaikan dengan kapasitas GPU
MAX_DISPLAY = 10
displayed = 0

# === TIMESTAMP UNTUK NAMA FILE === #
timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
output_dir = f"evaluation_results_{timestamp}"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
stratified_dataset

Dataset({
    features: ['instruction', 'input', 'output', 'topic_set', 'topics', '__index_level_0__'],
    num_rows: 100
})

In [ ]:
alpaca_prompt

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nJawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.\n\n### Input:\n{}\n\n### Response:\n{}'

In [ ]:
num = 1
for example in tqdm(stratified_dataset, desc="Evaluating Base Model"):
    # Gunakan prompt yang hanya menerima input (karena instruksi tetap)
    prompt = alpaca_prompt.format(example["input"], "")  # output dikosongkan untuk inferensi

    # Tokenisasi dan pindahkan ke GPU
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Inference (tanpa perhitungan gradien)
    with torch.no_grad():
        output = model.generate(
            **input_ids,
            max_new_tokens=1024,
            do_sample=True,
            top_p=0.9,
            temperature=0.7
        )

    # Decode output dan ekstrak bagian response-nya
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    response = decoded.split("### Response:")[-1].strip()

    # Cetak sampel jika masih kurang dari 10
    if num < 10:
        print(prompt)
        print()
        print('response:', response)
        print('-' * 80)

    # Simpan hasil untuk evaluasi
    generated_outputs.append(response)
    reference_outputs.append(example["output"])
    all_instructions.append("Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.")  # atau bisa juga diabaikan jika instruksi selalu sama
    all_inputs.append(example["input"])
    num += 1

Evaluating Base Model:   1%|          | 1/100 [00:35<58:29, 35.45s/it]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.

### Input:
Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1,5 kilo, tidak memiliki nafsu makan sama sekali, sering sendawa, nyeri di ulu hati dan perut bagian kanan. Kemarin saya juga sempat terlambat makan siang dan ma

Evaluating Base Model:   2%|▏         | 2/100 [00:55<42:45, 26.18s/it]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.

### Input:
solusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?

### Response:


response: konstipasi dapat disebabkan oleh beberapa hal berikut ini: kebiasaan mengabaikan keinginan untuk buang air besar kurang minum kurang konsumsi serat kurang aktif dan jarang berolahraga penurunan berat badan yang mendadak efek samping obat-obatan tertentu penyakit medis tertentu seperti penyakit crohn, penyumbatan usus, diabetes, gangguan tiroid, kanker usus, dsb. untuk memastikan penyebab dari konstipasi yang anda alami, maka sebaiknya anda memeriksakan diri secara langsung ke dokter. d

Evaluating Base Model:   3%|▎         | 3/100 [01:18<39:59, 24.74s/it]

...

Evaluating Base Model:   8%|▊         | 8/100 [03:21<36:00, 23.48s/it]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.

### Input:
sakit perut sebelah kiri bawah. Dok, saya mau tanya. Saya sudah menikah dan berusia 21 tahun. Sudah hampir 2minggu lebih perut sebelah kiri bawah sakit. Terhitung dari selesai saya haid tanggal 30 maret 2016. Sakitnya sampai ke pinggang dan tanda tanda nya seperti akan haid. Karena pada saya pun terasa nyeri dan cenderung lebih sensitif. Di minggu ketiga ini sakitnya terasa ngilu sampai ke paha kiri. Bahkan kadang panggul panggul pun sakit. Itu kenapa ya dok?apakah saya hamil? Terima kasih

### Response:


response: sebaiknya anda berkonsultasi dengan dokter spesialis kandungan. dokter akan menganalisis gejala yang anda alami dan melakukan pemeriks

Evaluating Base Model:   9%|▉         | 9/100 [03:25<26:20, 17.37s/it]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.

### Input:
Nyeri Ulu Hati Pasca Operasi Usus buntu. sudah 4 Bulan yang lalu saya di operasi usus buntu (Operasi Laparotomi) minggu minggu ini perut saya aga kurang enak dok. tadi pagi setelah BAB perut saya kram,melilit,perih....dan nyeri Ulu ati, nyerinya timbul tenggelam...tapi setelah minum obat maag (Omeprazol) sakitnya mereda tapi masih aga kurang enak perutnya itu penyebabnya apa ya dok? apa karena Maag atau ada iritasi di Usus? atau ada hubungannya dengan operasi usus buntu 4 bulan yang lalu?

### Response:


response: dokter akan melakukan pemeriksaan fisik dan tes penunjang jika diperlukan (seperti tes darah, usg, endoskopi) untuk menganalisis penyeb

Evaluating Base Model: 100%|██████████| 100/100 [40:27<00:00, 24.27s/it]


In [ ]:
# === Skor Evaluasi === #
P, R, F1 = bert_score.score(generated_outputs, reference_outputs, lang="id", device="cuda")
bleu_score = bleu.compute(predictions=generated_outputs, references=[[r] for r in reference_outputs])
rouge_score = rouge.compute(predictions=generated_outputs, references=reference_outputs)

timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
output_dir = f"eval_basemodel_{timestamp}"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# === Simpan Hasil Evaluasi === #
df = pd.DataFrame({
    "instruction": all_instructions,
    "input": all_inputs,
    "reference": reference_outputs,
    "generated": generated_outputs,
    "bert_precision": P.tolist(),
    "bert_recall": R.tolist(),
    "bert_f1": F1.tolist(),
})

df.to_csv(f"{output_dir}/fine_tuned_outputs.csv", index=False)

metrics = {
    "bert_score": {
        "precision": round(P.mean().item() * 100, 2),
        "recall": round(R.mean().item() * 100, 2),
        "f1": round(F1.mean().item() * 100, 2)
    },
    "bleu": round(bleu_score["bleu"] * 100, 2),
    "rouge": {
        "rouge1": round(rouge_score["rouge1"] * 100, 2),
        "rouge2": round(rouge_score["rouge2"] * 100, 2),
        "rougeL": round(rouge_score["rougeL"] * 100, 2)
    }
}

with open(f"{output_dir}/metrics.json", "w") as f:
    import json; json.dump(metrics, f, indent=4)

print("=== EVALUASI Fine tune MODEL SELESAI ===")
print(metrics)

=== EVALUASI Fine tune MODEL SELESAI ===
{'bert_score': {'precision': 69.6, 'recall': 69.19, 'f1': 69.35}, 'bleu': 4.55, 'rouge': {'rouge1': np.float64(30.25), 'rouge2': np.float64(6.82), 'rougeL': np.float64(15.66)}}


In [ ]:
! zip -r base_mode_4.zip /content/eval_basemodel_20250525_034008

updating: content/eval_basemodel_20250525_034008/ (stored 0%)
  adding: content/eval_basemodel_20250525_034008/metrics.json (deflated 46%)
  adding: content/eval_basemodel_20250525_034008/base_model_outputs.csv (deflated 77%)


In [ ]:
! zip -r ft_mode_4.zip /content/eval_basemodel_20250525_044500

  adding: content/eval_basemodel_20250525_044500/ (stored 0%)
  adding: content/eval_basemodel_20250525_044500/fine_tuned_outputs.csv (deflated 73%)
  adding: content/eval_basemodel_20250525_044500/metrics.json (deflated 47%)


In [ ]:
if True: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.24 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 34%|███▍      | 11/32 [00:00<00:01, 14.66it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [01:57<00:00,  3.68s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model-00001-of-00004.bin...
Unsloth: Saving model/pytorch_model-00002-of-00004.bin...
Unsloth: Saving model/pytorch_model-00003-of-00004.bin...
Unsloth: Saving model/pytorch_model-00004-of-00004.bin...
Done.


In [ ]:
if True: model.push_to_hub_merged("farwew/model_3", tokenizer, save_method = "merged_16bit", token = "hf_REMOVED_BY_SCRIPT")

Unsloth: You are pushing to hub, but you passed your HF username = farwew.
We shall truncate farwew/model_3 to model_3


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.21 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [01:33<00:00,  2.91s/it]


Unsloth: Saving tokenizer...

  0%|          | 0/1 [00:00<?, ?it/s]

...

pytorch_model-00003-of-00004.bin:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

pytorch_model-00004-of-00004.bin:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

pytorch_model-00002-of-00004.bin:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

pytorch_model-00001-of-00004.bin:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/farwew/model_3
